### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..


In [13]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.tools import tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = True

### Start with our Message class


In [14]:
@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime


In [15]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start()

### Let's reintroduce a tool


In [16]:
serper = GoogleSerperAPIWrapper()


@tool
def langchain_serper(query: str) -> str:
    """Useful for when you need to search the internet."""
    return serper.run(query)


autogen_serper = LangChainToolAdapter(langchain_serper)

In [17]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents


In [18]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(
            name,
            model_client=model_client,
            tools=[autogen_serper],
            reflect_on_tool_use=True,
        )

    @message_handler
    async def handle_my_message_type(
        self, message: Message, ctx: MessageContext
    ) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages(
            [text_message], ctx.cancellation_token
        )
        return Message(content=response.chat_message.content)


class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(
            name,
            model_client=model_client,
            tools=[autogen_serper],
            reflect_on_tool_use=True,
        )

    @message_handler
    async def handle_my_message_type(
        self, message: Message, ctx: MessageContext
    ) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages(
            [text_message], ctx.cancellation_token
        )
        return Message(content=response.chat_message.content)


class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(
        self, message: Message, ctx: MessageContext
    ) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(
            content=result + "\n\n## Decision:\n\n" + response.chat_message.content
        )


In [ ]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:
    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:
    # Creates 3 different runtimes, one for each agent. Each runtime can be run on a different machine.
    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")


In [20]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [21]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some pros of using AutoGen in your new AI Agent project:

1. **Scalability**: AutoGen’s modular and extensible framework allows for the creation of scalable systems, making it easier to accommodate future growth and complexity.

2. **Ease of Use**: The platform offers integrated observability and debugging tools, which simplify the monitoring and control of agent workflows, enhancing overall development efficiency.

These features can significantly improve the versatility and manageability of your AI project. 

TERMINATE

## Cons of AutoGen:
Here are some cons of using AutoGen for an AI Agent project:

1. **Documentation Issues**: The documentation is often criticized for being hard to read and lacking sufficient examples, which can make it challenging for developers to effectively implement and utilize the framework.

2. **Functional Limitations**: Certain features, such as structured outputs, may not work as intended, leading to potential frustrations during development and limiting the capabilities of the agents.

3. **Lack of Differentiation**: AutoGen may struggle to differentiate itself from similar applications like AG2, which could affect its appeal and support within the developer community.

4. **Complexity of Implementation**: Due to the difficulties in documentation and potential issues with functionality, implementing AutoGen can become a complex undertaking, affecting productivity and increasing development time.

These factors may make it less suitable for your project depending on your specific needs and the level of support you're looking for. 

TERMINATE



## Decision:

Based on the pros and cons presented by the research team, I would recommend **not to use AutoGen for the project**. 

**Rationale**: While AutoGen offers scalability and ease of use with integrated tools, the significant drawbacks related to documentation quality, functional limitations, and implementation complexity pose serious risks for the project's development timeline and overall efficiency. The potential frustrations and challenges could outweigh the benefits, especially if the project requires a reliable and straightforward framework for implementation.

TERMINATE

In [22]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [23]:
await host.stop()